In [4]:
from selenium.webdriver.firefox.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import time

In [5]:
db_connection = sqlite3.connect("DeepDao.db")
cursor = db_connection.cursor()

In [6]:
driver = webdriver.Firefox()
driver.get("https://messari.io/governor/daos")

def main():
        WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.jss26:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > button:nth-child(2)'))).click()
        
        WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.MuiTableBody-root > tr:nth-child(1)')))

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(4)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        

        dao_table = soup.find("tbody", class_="MuiTableBody-root css-fo7z8p")
        dao_row = dao_table.findChildren("tr", class_="MuiTableRow-root css-1hifdwt")

        type_list = ['Social / Community', 'Investment', 'Protocol', 'Product', 'Service', 'Grants', 
        'Media', 'Impact', 'Collector']

        i = 0 
    
        for dao in dao_row:
            raw_data = dao.findChildren("span")
            data_list = [i.string for i in raw_data]
            dao_name = data_list[0]
            if data_list[2] in type_list:
                tag = data_list[3]
            else:
                tag = data_list[2]

            insert_vals = f"""
                UPDATE Governance SET
                `Type`= '{data_list[1]}',
                `Tag`= '{tag}'
                WHERE EXISTS (SELECT * FROM Governance WHERE DAO='{dao_name}');
                """
            cursor.execute(insert_vals)
            # select = f"SELECT * FROM Governance WHERE DAO = '{dao_name}';"
            # cursor.execute(select)
            # print(cursor.fetchall())
        db_connection.commit()
            # i+=1
        db_connection.close()
        

main()
driver.quit()